In [4]:
import pickle

with open(r"C:\Users\jdeblander\LiveProjects\Image classification\train_test_val_array.pkl", "rb") as f:
    data = pickle.load(f)


In [5]:
import numpy as np
def reshape_array(dataset):

    nb_imgs = dataset.shape[0]
    height = dataset.shape[1]
    width = dataset.shape[2]
    channels = dataset.shape[3]

    return np.reshape(dataset, (nb_imgs, height*width*channels))




In [6]:
train = reshape_array(data[0])
test = reshape_array(data[2])
valid = reshape_array(data[4])

In [7]:
print(train.shape)
print(test.shape)
print(valid.shape)

(2400, 30000)
(800, 30000)
(800, 30000)


In [8]:
"""def assign_tag(tags):

    # Swift = 0
    # WagonR = 1
    vals = {
        'swift' : 0,
        'wagonr' : 1
    }


    for i,tag in enumerate(tags):
        tags[i] = vals[tag]

    return tags
"""
from sklearn.preprocessing import label_binarize

train_tag = label_binarize(data[1], classes= ['swift', 'wagonr']).ravel()
test_tag = label_binarize(data[3], classes= ['swift', 'wagonr']).ravel()
valid_tag = label_binarize(data[5], classes= ['swift', 'wagonr']).ravel()


In [9]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

names = [
    "QDA",
    "Nearest Neighbors",
    "Decision Tree",
    "Random Forest",
    "Linear SVM",
    "RBF SVM",
    "Naive Bayes"
]

In [10]:
classifiers = [
    QuadraticDiscriminantAnalysis(),
    KNeighborsClassifier(3),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(kernel= 'linear', C = 0.02),
    SVC(),
    GaussianNB()
]

In [11]:
scores_list = []
precision_list, recall_list, fscore_list = [], [], []
train_time_list = []

from time import time
# Metrics
from sklearn.metrics import precision_recall_fscore_support

# iterate over classifiers
for name, clf in zip(names, classifiers):
    t1 = time()
    clf.fit(train, train_tag)
    train_time = time() - t1
    print(f"Train done for classifier [{name}] done in {train_time} secs")

    t1 = time()
    score = clf.score(test, test_tag)
    print(f"Accuracy eval done for classifier [{name}] done in {time() - t1} secs")

    t1 = time()
    y_pred = clf.predict(test)
    precision, recall, fscore, _ = precision_recall_fscore_support(test_tag, y_pred, average='weighted')
    print(f"Precision, recall and f1 eval done for classifier [{name}] done in {time() - t1} secs")
    print("----" * 10)

    scores_list.append(score)
    precision_list.append(precision)
    recall_list.append(recall)
    fscore_list.append(fscore)
    train_time_list.append(train_time)

C:\Users\jdeblander\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Train done for classifier [QDA] done in 12.591502666473389 secs
Accuracy eval done for classifier [QDA] done in 2.1792080402374268 secs
Precision, recall and f1 eval done for classifier [QDA] done in 2.34141206741333 secs
----------------------------------------
Train done for classifier [Nearest Neighbors] done in 15.188425540924072 secs
Accuracy eval done for classifier [Nearest Neighbors] done in 114.89944171905518 secs
Precision, recall and f1 eval done for classifier [Nearest Neighbors] done in 110.52537608146667 secs
----------------------------------------
Train done for classifier [Decision Tree] done in 67.38866448402405 secs
Accuracy eval done for classifier [Decision Tree] done in 0.08894848823547363 secs
Precision, recall and f1 eval done for classifier [Decision Tree] done in 0.09299945831298828 secs
----------------------------------------
Train done for classifier [Random Forest] done in 24.844051837921143 secs
Accuracy eval done for classifier [Random Forest] done in 0.

In [12]:
import pandas as pd

data = {
    'Classifier' : names,
    'Precision' : precision_list,
    'Recall' : recall_list,
    'F1-score' : fscore_list,
    'Training time' : train_time_list
}

pd.DataFrame(data)

,Classifier,Precision,Recall,F1-score,Training time
0,QDA,0.551088,0.54750,0.539414,12.591503
1,Nearest Neighbors,0.632619,0.63250,0.632417,15.188426
2,Decision Tree,0.605000,0.60500,0.605000,67.388664
3,Random Forest,0.648751,0.64875,0.648749,24.844052
4,Linear SVM,0.621421,0.62125,0.621117,219.502551
5,RBF SVM,0.664293,0.66375,0.663472,223.758698
6,Naive Bayes,0.588673,0.58750,0.586132,1.493067


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid']
    }

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(valid, valid_tag)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=  19.9s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.8s remaining:    0.0s


[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=  20.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=  20.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=  20.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=  19.7s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=  18.8s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=  19.0s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=  19.4s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] .

[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 77.2min finished


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             verbose=2)

In [14]:
print(grid.best_estimator_)

SVC(C=1, gamma=0.001)


In [15]:
clf = SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
t1 = time()
clf.fit(train, train_tag)
train_time = time() - t1

In [17]:
print(f"Train done for classifier done in {train_time} secs")

t1 = time()
score = clf.score(test, test_tag)
print(f"Accuracy eval done for classifier done in {time() - t1} secs")

t1 = time()
y_pred = clf.predict(test)
precision, recall, fscore, _ = precision_recall_fscore_support(test_tag, y_pred, average='weighted')
print(f"Precision, recall and f1 eval done for classifier done in {time() - t1} secs")

Train done for classifier done in 270.37767720222473 secs
Accuracy eval done for classifier done in 82.96184659004211 secs
Precision, recall and f1 eval done for classifier done in 75.02700614929199 secs


In [19]:
print(score, precision, recall, fscore)

0.67625 0.6764982005945862 0.67625 0.6761361416122855
